In [6]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'key.json'

In [10]:
# %pip install db-dtypes google-cloud-bigquery

In [147]:
from google.cloud import bigquery

client = bigquery.Client()

# Perform a query.
QUERY = (
    'SELECT * FROM `bigquery-public-data.usa_names.ua_1910_2013` '
    'WHERE state = "TX" '
    'LIMIT 100')
QUERY1 = (
    'SELECT * FROM `bigquery-public-data.usa_names.usa_1910_2013` '
    'WHERE state = "TX" '
    'LIMIT 100')
query_job = client.query(QUERY)  # API request
query_job1 = client.query(QUERY1)  # API request
# rows = query_job.result() # Waits for query to finish
# for i in dir(query_job):
#     print(i)
table_reference = query_job1.destination

# Fetch table metadata to get the actual table name
table = client.get_table(table_reference)

# Get the actual table name
table_name = table.table_id
table_name

'anon7196bec97a3f6c6fe9bafb93d642dc9287e75f8329055df372ebecc67eec7b9c'

In [135]:
print(query_job.query)
print(query_job1.destination)
print(query_job.exception)
print(query_job1.exception)

SELECT * FROM `bigquery-public-data.usa_names.ua_1910_2013` WHERE state = "TX" LIMIT 100
test-bigquery-410304._6aa11130c003b357c3f134d7582a8ad13886f06d.anon7196bec97a3f6c6fe9bafb93d642dc9287e75f8329055df372ebecc67eec7b9c
<bound method PollingFuture.exception of QueryJob<project=test-bigquery-410304, location=US, id=c2e1944a-9fdd-4173-8434-31b6d9c98b04>>
<bound method PollingFuture.exception of QueryJob<project=test-bigquery-410304, location=US, id=7bd44343-5e4b-4e89-9cb0-3bf47d314d7d>>


In [163]:
from flask import Flask, request, render_template, flash, redirect, url_for, request,jsonify
from urllib.parse import unquote, quote
from google.cloud import bigquery
import os, pandas as pd
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'key.json'

app = Flask(__name__)

class BigQueryClientSingleton:
    _instance = None

    def __new__(cls):
        if cls._instance is None:
            cls._instance = super(BigQueryClientSingleton, cls).__new__(cls)
            cls._instance.client = bigquery.Client()
        return cls._instance

    @classmethod
    def get_instance(cls):
        if cls._instance is None:
            cls._instance = cls()
        return cls._instance

    @classmethod
    def close_instance(cls):
        if cls._instance is not None and hasattr(cls._instance, 'client'):
            cls._instance.client.close()
            cls._instance = None

@app.route("/")
def home():
    BigQueryClientSingleton.get_instance()
    return f"Hello, welcome to bigquery api app on flask! "

@app.route("/list_datasets/<param>")
def list_datasets(param):
    try:
        project_id = param
        datasets = list(BigQueryClientSingleton.get_instance().client.list_datasets(project=project_id))
        if len(datasets) > 0:
            dataset_info_list = [
                {
                    "dataset_id": dataset.dataset_id,
                    "completepath": f'{project_id}.{dataset.dataset_id}'
                }
                for dataset in datasets
            ]

            return jsonify({
                "count": len(datasets),
                "project_id": project_id,
                "results": dataset_info_list
            })
        return jsonify({"error": f"No datasets found in the project: {project_id}"}), 404
    except Exception as e:
        return jsonify({"error": str(e)}), 500
    finally:
        close_instance()

@app.route("/list_tables/<param>")
def list_tables(param):
    try:
        project_id, dataset_id = param.split(".")        
        # Construct the dataset reference
        dataset_ref = BigQueryClientSingleton.get_instance().client.dataset(dataset_id, project=project_id)
        # Get a list of tables in the dataset
        tables = list(client.list_tables(dataset_ref))
        if len(tables) > 0:

            return jsonify({
                "count": len(tables),
                "project_id": project_id,
                "dataset_id": dataset_id,
                "results": [{"table": table.table_id,"full_table": f'{project_id}.{dataset_id}.{table.table_id}','table_schema':[{"name": field.name, "type": field.field_type} for field in BigQueryClientSingleton.get_instance().client.get_table(table.reference).schema]} for table in tables]
            })
        return jsonify({"error": f"No DataSets found in the project: {project_id} under dataset: {dataset_id}"}), 404
    except Exception as e:
        return jsonify({"error": str(e)}), 500
    finally:
        close_instance()

@app.route("/<sql>")
def run_query(sql):
    # sql = unquote(sql)
    try:
        client_instance = BigQueryClientSingleton.get_instance().client
        query_job = client_instance.query(sql)
        # Handle errors
        if query_job.errors:
            return jsonify({"error": query_job.errors}), 404
        query_result = query_job.result().to_dataframe().to_dict()
        result_list = [dict(row) for row in query_result]
        return jsonify({
            "count" : len(result_list),
            "input_query" : query_job.query,
            "results" : result_list
        })
    except Exception as e:
        return jsonify({"error": str(e)}), 500        
    finally:
        close_instance()

@app.route("/close")
def close_instance():
    BigQueryClientSingleton.close_instance()
    return f"connection closed"

@app.route("/error")
def error():
    BigQueryClientSingleton.close_instance()
    error_message = request.args.get('err', 'An error occurred.')
    if error_message == 'An error occurred.':
        return f"Unfortunately {error_message}"
    return f"Error Occured, Message: {error_message}"
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [08/Jan/2024 11:28:35] "GET /SELECT%20*%20FRoM%20bigquery-public-data.austin_311.311_service_requests%20LIMIT%202 HTTP/1.1" 500 -
127.0.0.1 - - [08/Jan/2024 11:28:39] "GET /SELECT%20*%20FRoM%20bigquery-public-data.austin_311.311_service_requests%20LIMIT%202 HTTP/1.1" 200 -


In [ ]:
# # ----------------------------------- Main Code --------------------------------------------
# from flask import Flask, request, render_template, flash, redirect, url_for, request,jsonify
# from urllib.parse import unquote, quote
# from google.cloud import bigquery
# import os, pandas as pd
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'key.json'

# app = Flask(__name__)

# class BigQueryClientSingleton:
#     _instance = None

#     def __new__(cls):
#         if cls._instance is None:
#             cls._instance = super(BigQueryClientSingleton, cls).__new__(cls)
#             # Create the BigQuery client here
#             cls._instance.client = bigquery.Client()
#         return cls._instance

#     @classmethod
#     def get_instance(cls):
#         if cls._instance is None:
#             cls._instance = cls()
#         return cls._instance

#     @classmethod
#     def close_instance(cls):
#         if cls._instance is not None and hasattr(cls._instance, 'client'):
#             cls._instance.client.close()
#             cls._instance = None

# @app.route("/")
# def home():
#     BigQueryClientSingleton.get_instance()
#     return f"Hello, welcome to bigquery api app on flask! "

# @app.route("/list_datasets/<param>")
# def list_datasets(param):
#     try:
#         datasets = list(BigQueryClientSingleton.get_instance().client.list_datasets(project=param))
#         if len(datasets) > 0:
#             dataset_info = {
#                 "count":len(datasets),
#                 "project_id":param,
#                 "dataset_id": [],
#                 "completepath": []
#             }

#             # Fill the DataFrame with dataset information
#             for dataset in datasets:
#                 dataset_info["dataset_id"].append(dataset.dataset_id)
#                 dataset_info["completepath"].append(f'{project_id}.{dataset.dataset_id}')

#             # Create a DataFrame from the dictionary and return 
#             return pd.DataFrame(dataset_info).to_json(orient='records')

#         return redirect(url_for('error')+ f'?err={quote(str(f"No DataSets found! in the project: {param}"))}')

#     except Exception as e:
#         # Redirect to Error Page
#         return redirect(url_for('error')+ f'?err={quote(str(e))}')
    
#     finally:
#         close_instance()


# @app.route("/list_tables/<param>")
# def list_tables(param):
#     try:

#         project_id, dataset_id = param.split(".")
        
#         # Construct the dataset reference
#         dataset_ref = BigQueryClientSingleton.get_instance().client.dataset(dataset_id, project=project_id)

#         # Get a list of tables in the dataset
#         tables = list(client.list_tables(dataset_ref))
    
#         if len(tables) > 0:
 
#             table_info_list = [{"table": f'{project_id}.{dataset_id}.{table.table_id}'} for table in tables]

#             # Create a DataFrame from the list of dictionaries
#             df_result = pd.DataFrame(table_info_list)
            
#             # Create a result dictionary for the final DataFrame
#             result_dict = {
#                 "count": len(tables),
#                 "project_id": project_id,
#                 "dataset_id": dataset_id,
#                 "result": df_result.to_dict(orient='records')
#             }

#             # Convert the result dictionary to JSON
#             json_response = pd.Series(result_dict).to_json(default_handler=str)
            

#             return json_response

#         return redirect(url_for('error')+ f'?err={quote(str(f"No DataSets found! in the project: {project_id} under dataset: {dataset_id}"))}')

#     except Exception as e:
#         # Redirect to Error Page
#         return redirect(url_for('error')+ f'?err={quote(str(e))}')
    
#     finally:
#         close_instance()

# @app.route("/<sql>")
# def run_query(sql):
#     # sql = unquote(sql)
#     try:
#         client_instance = BigQueryClientSingleton.get_instance().client
#         query_job = client_instance.query(sql)
#         # Handle errors
#         if query_job.errors:
#             # Redirect to Error Page
#             return redirect(url_for('error')+ f'?err={quote(str(query_job.errors))}')

#         rows = query_job.result().to_dataframe()

#         return rows.to_json(orient='records')

#     except Exception as e:
#         # Redirect to Error Page
#         return redirect(url_for('error')+ f'?err={quote(str(e))}')
    
#     finally:
#         close_instance()

# @app.route("/close")
# def close_instance():
#     BigQueryClientSingleton.close_instance()
#     return f"connection closed"

# @app.route("/error")
# def error():
#     BigQueryClientSingleton.close_instance()
#     error_message = request.args.get('err', 'An error occurred.')
#     if error_message == 'An error occurred.':
#         return f"Unfortunately {error_message}"
#     return f"Error Occured, Message: {error_message}"
# app.run()
# # ------------------------------------------------------------------------------------------

In [4]:
from ngrok import ngrok
token  = '2adOpP183TcTfwi1rhZI4FMbvD7_47TYVnAj2bWQ62ekN7vSA'
ngrok.set_auth_token(token)
listener = ngrok.forward("localhost:5000")

In [5]:
listener.get_name

<function Task.get_name()>

In [23]:
# listener.url

AttributeError: '_asyncio.Task' object has no attribute 'url'

In [ ]:
ngrok.disconnect()

<Task pending name='Task-33' coro=<wrap() running at ngrok_wrapper:6>>

In [224]:
from urllib.parse import quote

# ...

# Example: Constructing a URL with an encoded SQL query
sql_query = "SELECT name FROM bigquery-public-data.usa_names.usa_1910_2013 WHERE state = 'TX' LIMIT 100"
encoded_query = quote(sql_query)
encoded_query

'SELECT%20name%20FROM%20bigquery-public-data.usa_names.usa_1910_2013%20WHERE%20state%20%3D%20%27TX%27%20LIMIT%20100'

In [225]:
# from flask import Flask, request, render_template, flash, redirect, url_for

# from google.cloud import bigquery

# app = Flask(__name__)
# client_instance = None

# class BigQueryClientSingleton:
#     _instance = None

#     def __new__(cls):
#         if cls._instance is None:
#             cls._instance = super(BigQueryClientSingleton, cls).__new__(cls)
#             # Create the BigQuery client here
#             cls._instance.client = bigquery.Client()
#         return cls._instance

#     @classmethod
#     def get_instance(cls):
#         if cls._instance is None:
#             cls._instance = cls()
#         return cls._instance

#     @classmethod
#     def close_instance(cls):
#         if cls._instance is not None and hasattr(cls._instance, 'client'):
#             cls._instance.client.close()
#             cls._instance = None

# @app.route("/")
# def home():
#     global client_instance
#     client_instance = BigQueryClientSingleton.get_instance()
#     return f"Hello, Flask! {client_instance}"

# @app.route("/close")
# def close_instance():
#     BigQueryClientSingleton.close_instance()
#     return "Connection closed"

# @app.route("/error")
# def error_handler():
#     BigQueryClientSingleton.close_instance()
#     return "Connection closed"

# @app.route("/<sql>")
# def run_query(sql):
#     global client_instance
#     user_query = sql
#     query_job = client_instance.client.query(user_query)
#     print(f"error message: {query_job.errors}")
#     # Handle errors if needed
#     rows = query_job.result().to_dataframe()
#     return rows.to_dict()

# if __name__ == "__main__":
#     app.run()

In [ ]:
"SELECT name FROM bigquery-public-data.usa_names.usa_1910_2013 WHERE state = 'TX' LIMIT 100"

In [67]:
from google.cloud import bigquery

# Set up the BigQuery client
client = bigquery.Client()

# Replace 'your_project' with your actual project name
project_id = 'bigquery-public-data'
# project_id = 'test-bigquery-410304'
# project_id = ''
# Get a list of datasets in the project
datasets = list(client.list_datasets(project=project_id))

# Print the dataset names
# for dataset in datasets:
#     print(dataset.dataset_id)


In [70]:
from google.cloud import bigquery

# Set up the BigQuery client (the client will try to infer the project ID)
client = bigquery.Client()

# Replace 'your_dataset' with your actual dataset name
dataset_id = 'bigquery-public-data.austin_bikeshare'

# Construct the dataset reference
dataset_ref = client.dataset(dataset_id,project_id)

print(dataset_ref)
# Get a list of tables in the dataset
tables = list(client.list_tables(dataset_ref))

# # Print the table names
# for table in tables:
#     print(table.table_id)


bigquery-public-data.austin_bikeshare
